In [ ]:
#connect to influxdb on localhost
#token whVmtiLViagPA8zCpz4-ItfX56GPhoGMUg4s9u-kx7fXmZTNcVE9xWNbLoTXB0c347vMG8vUxxIAKDPJdsFO6A==
#org dsi
#bucket s-p-500
#bucket_id 5ab39a6efe08bcae
#url http://localhost:8086

from influxdb_client import InfluxDBClient, Point, WritePrecision
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate a Token from the "Tokens Tab" in the UI
token = "whVmtiLViagPA8zCpz4-ItfX56GPhoGMUg4s9u-kx7fXmZTNcVE9xWNbLoTXB0c347vMG8vUxxIAKDPJdsFO6A=="
org = "dsi"
bucket = "s-p-test"
client = InfluxDBClient(url="http://localhost:8086", token=token)

write_api = client.write_api(write_options=SYNCHRONOUS)

In [ ]:
p = Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)
write_api.write(bucket=bucket, org=org, record=p)

#close connection
client.close()

In [ ]:
#write for 500 points

for i in range(10):
    p = Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)
    write_api.write(bucket=bucket, org=org, record=p)


In [ ]:
# Querying data
query_api = client.query_api()
query = 'from(bucket:"s-p-test")\
|> range(start: -10m)\
|> filter(fn:(r) => r._measurement == "my_measurement")\
|> filter(fn:(r) => r.location == "Prague")\
|> filter(fn:(r) => r._field == "temperature")'

result = query_api.query(query, org=org)

In [ ]:
results = []
for table in result:
  for record in table.records:
    results.append((record.get_field(), record.get_value()))

print(results)

In [ ]:
#create dataframe wich memics a timeseries of a stock
import pandas as pd
import numpy as np
import datetime
import random

# Create a dataframe with a column
df = pd.DataFrame({'value': np.random.randn(1000)})
# Add a column with a datetime index
df['date'] = pd.date_range('1/1/2000', periods=1000)
# Set the datetime column as the index
df = df.set_index('date')
# Create a column with a random walk
df['value'] = df['value'].cumsum()


print(df)
#plot df
import matplotlib.pyplot as plt
plt.plot(df.index, df['value'])
plt.show()

In [ ]:
import pandas as pd

records = pd.DataFrame(
    data=[
        ["0", 25.3, 1657729063],
    	  ["0", 25.4, 1657729078],
    	  ["0", 25.2, 1657729093],
    ],
    columns=["core", "temp", "timestamp"],
)

records

In [ ]:

with InfluxDBClient.from_config_file("config.toml") as client:
    with client.write_api() as writer:
        writer.write(
        	bucket="testing",
        	record=records,
        	data_frame_measurement_name="cpu",
        	data_frame_tag_columns=["core"],
        )

In [ ]:
token = "whVmtiLViagPA8zCpz4-ItfX56GPhoGMUg4s9u-kx7fXmZTNcVE9xWNbLoTXB0c347vMG8vUxxIAKDPJdsFO6A=="
org = "dsi"
bucket = "s-p-test"
client = InfluxDBClient(url="http://localhost:8086", token=token)

write_api = client.write_api(write_options=SYNCHRONOUS)

write_api.write(
        org="dsi",
        bucket="testing",
        record=records,
        data_frame_measurement_name="cpu",
        data_frame_tag_columns=["core"]
    )

#close connection
client.close()

In [ ]:
#get json from https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=5LZM13OZQMZFPU92

import requests
import json
import pandas as pd
import numpy as np
import datetime

url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=5LZM13OZQMZFPU92"
response = requests.get(url)
data = response.json()

#extract "Meta Data" and every "Time Series (5min)" entry
meta_data = data["Meta Data"]
time_series = data["Time Series (5min)"]

#convert time_series to dataframe
df = pd.DataFrame.from_dict(time_series, orient='index')

# #print every row

# for index, row in df.iterrows():
#     print(row)

#sort dataframe by index
df = df.sort_index()
df


In [ ]:
# Raw Package
import numpy as np
import pandas as pd



#Data viz
import plotly.graph_objs as go

#Data Source
import yfinance as yf

#Interval required 1 minute
data = yf.download(tickers='IBM', period='1d', interval='1m')

#declare figure
fig = go.Figure()

#Candlestick
fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'], name = 'market data'))

# Add titles
fig.update_layout(
    title='Uber live share price evolution',
    yaxis_title='Stock Price (USD per Shares)')

# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#Show
fig.show()

In [ ]:
data = yf.download(tickers='IBM', period='1d', interval='1m')

In [ ]:
#convert dataframe to json
a = df.to_json(orient='index')

In [ ]:
type(data)
type(a)

In [ ]:
test = pd.read_json(a, orient='index')
test

In [ ]:
import influxdb_client
from influxdb import DataFrameClient
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.write_api import ASYNCHRONOUS

#InfluxDB connection
bucket = "DSI_test"
#token ="yxk8_Or5qHZJxrhJE3SkAnTQSViQCsmrUoR0xPZd_0scy1T8FTuL1cKSTDKh1ft8Bqs3Zbt7Rwkys-FzajIVFQ=="
token ="GMbnHWGhM9p9t9mIjbc1I5KlWir8LJxBDkpMF0SiOa56f1nvLepCEN7iI_5-sR80FA8CvLmf_mHcy8Gc5XYwvA=="
org="dsi"
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)
#client = InfluxDBClient(url="http://172.19.0.2:8086", token=token, org=org)


In [ ]:
data = yf.download(tickers='IBM', period='1d', interval='1m')

In [ ]:
data

In [ ]:
with client.write_api(write_options=SYNCHRONOUS) as writer:
    writer.write(
        bucket=bucket,
        record=data,
        data_frame_measurement_name="key2",
        #data_frame_tag_columns=["symbol"],
        data_frame_field_columns=["Open", "High", "Low", "Close", "Adj Close", "Volume"]
        )

In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
data = yf.download(tickers="IBM", period='1d', interval='1m').to_json(orient = "index")


In [13]:
df = pd.read_json(data).T

NameError: name 'pd' is not defined

In [ ]:
df

In [ ]:
#set indx name of dataframe to Datetime
df.index.name = 'Datetime'

In [ ]:
df